# <a id='toc1_'></a>[Pacific dataviz Challenge 2025](#toc0_)
**The data processing notebook**

![Banner Image](<img/Roméo_et_juliette.png> "Roméo & Juliet play at the théâtre de la Cité in Oct.'25.") <br>Roméo et Juliette *(théâtre de la Cité - October '25; Photography: Christophe Raynaud de Lage)*

> This notebook converts the original Pacific Dataviz Challenge dataset: a (quirky) Excel export into cleaned CSV and JSON files suitable for a analysis and use as backend for web apps.

**Table of contents**<a id='toc0_'></a>    
- [Pacific dataviz Challenge 2025](#toc1_)    
  - [Genesis](#toc1_1_)    
    - [Imports and configuration](#toc1_1_1_)    
  - [Section 1 - Data consolidation and preparation](#toc1_2_)    
    - [Looking for sentinels](#toc1_2_1_)    
    - [Solid foundations: restructuring the data](#toc1_2_2_)    
      - [Geographic insights](#toc1_2_2_1_)    
    - [Mapping key domains](#toc1_2_3_)    
    - [Enrich with an indicator summary](#toc1_2_4_)    
    - [Generate a complete group×state×year grid and detect missing records](#toc1_2_5_)    
      - [Investigate missing data](#toc1_2_5_1_)    
  - [Section 2) The case of Group58 and Group18](#toc1_3_)    
    - [Group 58 - Transform wide-format-data to a long-format](#toc1_3_1_)    
    - [Group 18 - The case of inverse relationships](#toc1_3_2_)    
  - [Section 3) Prevalence filtering](#toc1_4_)    
  - [Section 4) Scaling raw values](#toc1_5_)    
  - [Section 5) Additional features](#toc1_6_)    
    - [Metrics for completeness and contributions/rankings](#toc1_6_1_)    
    - [Interactive report about data quality](#toc1_6_2_)    
    - [Export data to JSON](#toc1_6_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Genesis](#toc0_)

1) From a messy Excel spreadsheet:

![Banner Image](<img/excel.png> "Screenshot of an Excel spreadsheet.")

2) To neat, **exploitable** datasets:

CSV sample:

```csv
Group_id,Year,class_name,class_num,indicator_summary,Subregion_code,Subregion_name,State_code,State_name,Raw_values,Scaled_Values,value_contribution_to_class,State_rank_per_class,pct_missing_values
0,1,2022,Political Leadership and Regionalism,1,Total development aid,MEL,Melanesia,FJ,Fiji,561024940.0000,0.8507,0.0305,6,0.0000
```

JSON sample:

```json
[
    {
        "group_id": 1,
        "year": 2022,
        "class_name": "Political Leadership and Regionalism",
        "class_num": 1,
        "indicator_summary": "Total development aid",
        "subregion_code": "MEL",
        "subregion_name": "Melanesia",
        "state_code": "FJ",
        "state_name": "Fiji",
        "raw_values": 561024940.0,
        "scaled_values": 0.8506735359,
        "value_contribution_to_class": 0.0305291251,
        "state_rank_per_class": 6,
        "pct_missing_values": 0.0
    }
]
```


### <a id='toc1_1_1_'></a>[Imports and configuration](#toc0_)

In [1]:
# Install required libraries
#%pip install pandas numpy ipywidgets

In [2]:
import pandas as pd
import json
import numpy as np

from IPython.display import display, HTML
import ipywidgets as widgets

# Disable scientific notation
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## <a id='toc1_2_'></a>[Section 1 - Data consolidation and preparation](#toc0_)

> The processing pipeline is nearly fully automated; the first manual step was preparing the original Excel file by removing layout elements before loading it into a dataframe and prevent repeated openpyxl crashes.
>  The other manual intervention consist in enriching the dataset metadata with LLM-generated topic summaries [see this part](#enrich-with-an-indicator-summary).

Let's have a look at the raw data:

In [3]:
df = pd.read_excel('data/cleared_format_excel.xlsx', engine='openpyxl')
df.head()

,Unnamed: 0,(DF_BP50) Blue Pacific 2050 (all),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,(FREQ) Frequency: (A) Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,(EDUCATION) Education level,(EDUCATION) Education level,(EDUCATION) Education level,(_T) All education levels,(02) Pre-primary education,(1) Primary education,(2-3) Secondary education (lower and upper sec...,(2) Lower secondary education,(3) Upper secondary education,NaN
3,NaN,(GEO_PICT) Pacific Island Countries and territ...,(TIME_PERIOD) Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(INDICATOR) Indicator: (DC_TRF_TOTL) Political...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is not good. Erm.. This is.. bad! Data is poorly structured. It's time to achieve some wonders.

### <a id='toc1_2_1_'></a>[Looking for sentinels](#toc0_)

1) On a logical level, we use a "group" granularity where each group is a collection of records for multiple countries corresponding to a **unique** indicator.

Have a look back at the spreadsheet above: one group example is, *"total assistance for development, by recipent countries".*

2) The cell below is designed to organize and structure the data. Process a DataFrame to identify and group records based on predefined sentinel markers, such as `(INDICATOR)` and `(OCCUPATION)`. 
   - Have a look at the excel spreadsheet screenshot (again, yes! ;). **A grey cell** precedes each groups data and its composition is consistent: it always contains the same marker words.
   -  The algorithm will jump from grey-cell-content to grey-cell-content and isolate the records between them as belonging to the same indicator **group**. 

> How? These markers identify distinct sections of data we need to isolate by iterating through the specified column, checking if rows start with these flags, and collects full sentinel texts into a buffer. When all markers of a group are found (meaning the group is complete), it logs the group and resets for the next. Finally, it assigns group IDs back to the DataFrame and exports the collected metadata to a JSON file.


In [4]:
# Define the sentinel markers in order
sentinels = [
    "(INDICATOR)",
    "(OCCUPATION)",
    "(COMPOSITE_BREAKDOWN)",
    "(DISABILITY)",
]

group_id = 0
next_sentinel_idx = 0
sentinel_buffer = {}
groups_meta = []
group_ids = []

# Iterate over each row
for idx, cell in df["(DF_BP50) Blue Pacific 2050 (all)"].fillna("").items():
    text = cell.strip()

    # Check if it starts with the next sentinel
    if text.startswith(sentinels[next_sentinel_idx]):
        # Record the full sentinel text
        sentinel_buffer[next_sentinel_idx] = text

        # Move to look for the next sentinel
        next_sentinel_idx += 1

        # If we have collected all 4, that completes one group
        if next_sentinel_idx == len(sentinels):
            group_id += 1
            groups_meta.append({
                "group_number": group_id,
                "sentinels": [sentinel_buffer[i] for i in range(len(sentinels))]
            })
            # Reset for the next group
            next_sentinel_idx = 0
            sentinel_buffer.clear()

    # Assign current group_id (0 if group not started yet)
    group_ids.append(group_id)

# Attach the group column
df["Group_id"] = group_ids

json_out = json.dumps(groups_meta, indent=2)

with open("data/groups_metadata.json", "w", encoding="utf-8") as f:
    f.write(json_out)

### <a id='toc1_2_2_'></a>[Solid foundations: restructuring the data](#toc0_)

In order to use the dataset computationally. A consistent tabular structure is needed.

1) Clean the DataFrame by first dropping unnecessary columns. 
2) Then extract and convert the year from the `Unnamed: 2` column to an integer.
3) Regional information: The original column (containing the regional data) is renamed for clarity as `RegionFull`. The code identifies state names by looking for lines that start with a bullet point `"·"` marker. It creates new columns for `Subregion` and `State` based on the content of `RegionFull`. States are linked to their preceding subregions, and the code extracts codes for both subregions and states while removing any extraneous characters.
4)  Finally, filter the data into a new DataFrame, `df_data`, containing only relevant entries and columns. The result is a streamlined dataset.

In [5]:
# 1) Drop the unwanted columns
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 3'])

# 2) Clean up the year‐column (formerly Unnamed: 2)
# Remove the parentheses and cast to int.
df['Year'] = df['Unnamed: 2'] \
    .astype(str) \
    .str.extract(r'\)\s*(\d{4})') \
    .iloc[:, 0] \
    .astype('Int32')

# 3) Split out subregions and states from the “(DF_BP50) Blue Pacific 2050 (all)” column
#    We’ll call that column “RegionFull” for clarity
df = df.rename(columns={'(DF_BP50) Blue Pacific 2050 (all)': 'RegionFull'})

# Create empty columns
df['Subregion'] = pd.NA
df['State'] = pd.NA

current_sub = None

for idx, cell in df['RegionFull'].items():
    if pd.isna(cell):
        continue

    s = str(cell).lstrip()
    # Detect a state line by the leading dot “·”
    if s.startswith('·'):
        # It is a state
        # remove the bullet and any extra whitespace:
        name = s.lstrip('· ').strip()
        df.at[idx, 'State'] = name
        # carry down the most recent subregion if any
        if current_sub:
            df.at[idx, 'Subregion'] = current_sub

    else:
        # It is a subregion row
        name = s.strip()
        df.at[idx, 'Subregion'] = name
        current_sub = name
        # No state on the subregion row itself

df[['Subregion_code', 'Subregion_name']
   ] = df['Subregion'].str.split(' ', n=1, expand=True)
df['Subregion_code'] = df['Subregion_code'].str.strip('()')

df[['State_code', 'State_name']] = df['State'].str.split(' ', n=1, expand=True)
df['State_code'] = df['State_code'].str.strip('()')

df = df.drop(columns=['RegionFull', 'Unnamed: 2'])

df_data = df[df['State'].notna()].copy()
df_data = df_data.drop(columns=['Subregion', 'State'])

# Now df_data has columns: Year (int), Subregion, State, plus any other data columns.
df_data

,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Group_id,Year,Subregion_code,Subregion_name,State_code,State_name
9,561024940,NaN,NaN,NaN,NaN,NaN,NaN,1,2022,MEL,Melanesia,FJ,Fiji
10,572455900,NaN,NaN,NaN,NaN,NaN,NaN,1,2022,MEL,Melanesia,PG,Papua New Guinea
11,256630130,NaN,NaN,NaN,NaN,NaN,NaN,1,2022,MEL,Melanesia,SB,Solomon Islands
12,183972650,NaN,NaN,NaN,NaN,NaN,NaN,1,2022,MEL,Melanesia,VU,Vanuatu
14,90657960,NaN,NaN,NaN,NaN,NaN,NaN,1,2022,MIC,Micronesia,KI,Kiribati
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,0.3600,NaN,NaN,NaN,NaN,NaN,NaN,112,2024,POL,Polynesia,WS,Samoa
1971,0.3300,NaN,NaN,NaN,NaN,NaN,NaN,112,2022,POL,Polynesia,TO,Tonga
1972,0.3200,NaN,NaN,NaN,NaN,NaN,NaN,112,2024,POL,Polynesia,TO,Tonga
1973,0.2300,NaN,NaN,NaN,NaN,NaN,NaN,112,2022,POL,Polynesia,TV,Tuvalu


#### <a id='toc1_2_2_1_'></a>[Geographic insights](#toc0_)

Now we can extract the distinct States and subregions present within the dataset:

In [6]:
distinct_subregion_names = df_data['Subregion_name'].unique()

for subregion in distinct_subregion_names:
    print(f"Countries in {subregion}:")
    countries = df_data[df_data['Subregion_name']
                        == subregion]['State_name'].unique()
    print(countries)
    print()

Countries in Melanesia:
['Fiji' 'Papua New Guinea' 'Solomon Islands' 'Vanuatu' 'New Caledonia']

Countries in Micronesia:
['Kiribati' 'Marshall Islands' 'Micronesia (Federated States of)' 'Nauru'
 'Palau']

Countries in Polynesia:
['Niue' 'Samoa' 'Tonga' 'Tuvalu' 'French Polynesia' 'Cook Islands']



### <a id='toc1_2_3_'></a>[Mapping key domains](#toc0_)

Define first a list containing the seven key domains and create a mapping dictionary, `topic_ids`, that assigns each topic a unique numerical ID using an auto-incrementing pattern, starting from 1.

In [7]:
class_list = [
    "Political Leadership and Regionalism",
    "People-Centred Development",
    "Peace and Security",
    "Resources and Economic Development",
    "Climate Change and Disasters",
    "Ocean and Environment",
    "Technology and Connectivity"
]

# Build the dictionary with auto-incrementing IDs
topic_ids = {topic: idx for idx, topic in enumerate(class_list, start=1)}

print(topic_ids)

{'Political Leadership and Regionalism': 1, 'People-Centred Development': 2, 'Peace and Security': 3, 'Resources and Economic Development': 4, 'Climate Change and Disasters': 5, 'Ocean and Environment': 6, 'Technology and Connectivity': 7}


### <a id='toc1_2_4_'></a>[Enrich with an indicator summary](#toc0_)

**Important**

An `indicator_summary` key-value pair has been generated with the help of an LLM on top of the initial [JSON containing the group structures](#looking-for-sentinels). It summarizes the whole content of the `(INDICATOR)` value.

The purpose of this process is to enrich the dataset, `df_data`, by incorporating (manually in this case, but this could be automated) synthetically generated metadata that categorizes with **more granularity** the data based on their topics descriptions.

1) Begin by loading metadata from a JSON file, `groups_meta_augmented.json`, into a Python dictionary called `lookup`. Construct a DataFrame, `meta`, containing only the relevant fields: `Group_id`, `class_name`, and `indicator_summary`. 
2) Next, map each `class_name` to a numerical ID using the previously defined `topic_ids`. Finally, merge the `meta` DataFrame back into the original DataFrame, `df_data`, based on the `Group_id`, effectively **enriching** `df_data` with the additional summary information.

In [8]:
with open('data/groups_metadata_augmented.json') as f:
    lookup = json.load(f)

# 1) Build a small DataFrame from the JSON
# Only need the fields: group_number, class, indicator_summary
meta = pd.DataFrame([
    {
        "Group_id":          item["group_number"],
        "class_name":        item["class"],
        "indicator_summary": item["indicator_summary"]
    }
    for item in lookup
])

meta["class_num"] = meta["class_name"].map(topic_ids)

# 2) Merge back into original df
df_data = df_data.merge(
    meta,
    on="Group_id",
    how="left"
)

### <a id='toc1_2_5_'></a>[Generate a complete group×state×year grid and detect missing records](#toc0_)

The goal is to ensure coverage of all subregion-state combinations across the identified groups and years. This process highlights any missing data points that ought to facilitate further investigation or remediation.

1) Generate a master list of unique subregion-state combinations from `df_data` (this confirms no duplicates).
2) All unique combinations of `Group_id` and `Year` are extracted to represent the available groups and years.
3) **Create cartesian product:** a cartesian product is generated by merging the groups and master combinations, resulting in every possible combination of group-year and subregion-state.
4) Detect missing rows: an *anti-join* is performed against the original dataset to find any combinations present in the cartesian product that are missing from the actual data, flags are identified through the `_merge` indicator.
5) Extract missing keys: finally, isolate just the key columns from the missing data, thus allowing for targeted investigation of gaps in the dataset.

In [9]:
# 1) Identify the master list of subregion↔state combos
master = (
    df_data[[
        "Subregion_code",
        "Subregion_name",
        "State_code",
        "State_name"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 2) Identify all the (Group_id, Year) combos
groups = (
    df_data[["Group_id", "Year",	"class_name",	"indicator_summary",	"class_num"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 3) Make a cartesian product of every group×year with every master combo
groups["_tmpkey"] = 1
master["_tmpkey"] = 1

full = (
    groups
    .merge(master, on="_tmpkey")
    .drop(columns="_tmpkey")
)

# 4) Anti‐join full against the real data, but only on the key columns.
#    *don’t* drop the Unnamed… cols yet, only use the keys to detect missing rows.
merged = full.merge(
    df_data,
    on=[
        "Group_id",
        "Subregion_code", "Subregion_name",
        "State_code", "State_name","Year",	"class_name",	"indicator_summary",	"class_num"
    ],
    how="left",
    indicator=True
)

missing = merged[merged["_merge"] == "left_only"]

# If key‐columns necessary
missing_keys = missing[[
    "Group_id",
    "Subregion_code", "Subregion_name",
    "State_code", "State_name"
]]

#### <a id='toc1_2_5_1_'></a>[Investigate missing data](#toc0_)

In [10]:
merged.head()

,Group_id,Year,class_name,indicator_summary,class_num,Subregion_code,Subregion_name,State_code,State_name,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,_merge
0,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,FJ,Fiji,561024940,NaN,NaN,NaN,NaN,NaN,NaN,both
1,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,PG,Papua New Guinea,572455900,NaN,NaN,NaN,NaN,NaN,NaN,both
2,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,SB,Solomon Islands,256630130,NaN,NaN,NaN,NaN,NaN,NaN,both
3,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,VU,Vanuatu,183972650,NaN,NaN,NaN,NaN,NaN,NaN,both
4,1,2022,Political Leadership and Regionalism,Total development aid,1,MIC,Micronesia,KI,Kiribati,90657960,NaN,NaN,NaN,NaN,NaN,NaN,both


## <a id='toc1_3_'></a>[Section 2) The case of Group58 and Group18](#toc0_)

### <a id='toc1_3_1_'></a>[Group 58 - Transform wide-format-data to a long-format](#toc0_)

**Group 58** is special because its five education-related columns are stored as separate wide-format columns that need to be melted into rows (one per education level) and remapped into five new Group_id/indicator_summary values (581–585). All other groups are already in the desired long format.

![Screenshot Image](<img/education.png> "Screenshot of an Excel spreadsheet.")

Processing:

1) Restructure the DataFrame by renaming columns and isolating rows for a specific group (Group 58), followed by *melting multiple education-related columns into a long format*. 
2) Additionally, assign new group IDs and summaries based on education levels, and merge this transformed data with the remaining dataset.

Secundarily: Clean up the final DataFrame by dropping obsolete columns while making sure data types are correctly formatted.

In [11]:
# 1) Rename Unnamed:4 to values, drop Unnamed:10
df_grp = merged.rename(columns={'Unnamed: 4': 'values'}).drop(
    columns=['Unnamed: 10'])

# 2) Split off the group 58 rows
df58 = df_grp[df_grp['Group_id'] == 58].copy()
df_rest = df_grp[df_grp['Group_id'] != 58].copy()

# 3) Melt the five education columns
education_cols = {
    'Unnamed: 5': 'Pre-primary education',
    'Unnamed: 6': 'Primary education',
    'Unnamed: 7': 'Secondary education',
    'Unnamed: 8': 'Lower secondary education',
    'Unnamed: 9': 'Upper secondary education',
}

melted = (
    df58
    .melt(
        id_vars=[c for c in df58.columns
                 if c not in education_cols.keys()],
        value_vars=list(education_cols.keys()),
        var_name='education_col',
        value_name='val'
    )
    # Drop any rows where the cell was NaN
    # .dropna(subset=['values'])
)

# 4) Assign a new Group_id and indicator_summary based on the education level
group_mapping = {
    'Unnamed: 5': (581, 'Total development aid – Pre-primary'),
    'Unnamed: 6': (582, 'Total development aid – Primary'),
    'Unnamed: 7': (583, 'Total development aid – Secondary'),
    'Unnamed: 8': (584, 'Total development aid – Lower secondary'),
    'Unnamed: 9': (585, 'Total development aid – Upper secondary'),
}


def map_group(row):
    gid, summary = group_mapping[row['education_col']]
    row['Group_id'] = gid
    row['indicator_summary'] = summary
    return row


melted = melted.apply(map_group, axis=1)

# If need to reassign class_num etc., do that here.
# e.g. melted['class_num'] = melted['Group_id'].map(my_classnum_map)

# 5) Concatenate back together
df_streamlined = pd.concat([df_rest, melted], ignore_index=True)

# Now `df_final` has:
# - all the original groups (except 58)
# - for group 58 there are 5 new groups (581–585), one per education level,
#   with their values transposed into rows
# 1) merge the 'val' values into the 'values' column, preferring 'val' when it's not null
df_streamlined['values'] = df_streamlined['val'].combine_first(
    df_streamlined['values'])

int_cols = ['Group_id', 'class_num', 'Year']
float_cols = ['values']

df_streamlined[int_cols] = df_streamlined[int_cols].astype(int)
# df_streamlined[float_cols] = df_streamlined[float_cols].astype(float)

# 3) Drop the unwanted columns
cols_to_drop = [
    'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
    'Unnamed: 8', 'Unnamed: 9', '_merge',
    'education_col', 'val'
]
df_streamlined = df_streamlined.drop(columns=cols_to_drop)
df_streamlined['Raw_values'] = df_streamlined['values']

print(df_streamlined.Group_id.unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 581 582 583 584 585]


And this is what we have so far:

In [12]:
df_streamlined.head(3)

,Group_id,Year,class_name,indicator_summary,class_num,Subregion_code,Subregion_name,State_code,State_name,values,Raw_values
0,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,FJ,Fiji,561024940,561024940
1,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,PG,Papua New Guinea,572455900,572455900
2,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,SB,Solomon Islands,256630130,256630130


Much better! 👌

### <a id='toc1_3_2_'></a>[Group 18 - The case of inverse relationships](#toc0_)

**Group 18** is about the tuberculosis (TB) incidence rate. The higher it is, the worse the health outcome. This reflects an inverse relationship that deviates from the patterns observed in the rest of the dataset and will introduce noise into the synthetic scores computed later.

- A reciprocal transformation adresses inverse relationships directly:

In [13]:
# Apply reciprocal transformation (only non-zero values)
df_streamlined.loc[(df_streamlined['Group_id'] == 18) & (df_streamlined['values'] != 0), 'values'] = 1 / \
    df_streamlined.loc[(df_streamlined['Group_id'] == 18) & (
        df_streamlined['values'] != 0), 'values']

In [14]:
# or drop group 
#df_streamlined = df_streamlined[df_streamlined['Group_id'] != 18]

## <a id='toc1_4_'></a>[Section 3) Prevalence filtering](#toc0_)

Like many reports involving manual (international) data collection, reporting practices vary across different organizations. The result is a strong data imbalance among various groups, years, and key indicators. To make sense8 of this disparity, keep only the groups that have a sufficient number of observations.

1) Identify and filter out groups from the DataFrame `df_streamlined` that have more than 12 missing values in the values column. This means we keep only groups that are at least 25% complete (a topic group contains 16 distinct States).
2) Keep only years **2022** and **2023** as they are the most complete.

In [15]:
missings = df_streamlined.groupby('Group_id')['values'].apply(lambda s: s.isna().sum())

# 1) Identify groups to drop
to_drop = missings[missings > 12].index

# 2) Filter them out
df_clean = df_streamlined[~df_streamlined['Group_id'].isin(to_drop)].copy()
# Keep only rows where Year is 2022 or 2023
df_clean = df_clean[df_clean['Year'].isin([2022, 2023])]

print(f"Dropped {len(to_drop)} groups: {list(to_drop)}")

Dropped 59 groups: [2, 3, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 30, 47, 50, 52, 56, 57, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 74, 76, 77, 79, 80, 81, 82, 83, 84, 85, 88, 95, 100, 102, 103, 104, 105, 106, 107, 109, 110, 583, 584, 585]


## <a id='toc1_5_'></a>[Section 4) Scaling raw values](#toc0_)

The purpose of this project is to compute an indicator that effectively summarizes complex data into a single, interpretable value **(a synthesis score)**. To achieve this, it is crucial to address the challenge of handling mixed values across broad range scales.

First, check types and cast features:

In [16]:
df_clean.dtypes

Group_id              int64
Year                  int64
class_name           object
indicator_summary    object
class_num             int64
Subregion_code       object
Subregion_name       object
State_code           object
State_name           object
values               object
Raw_values           object
dtype: object

In [17]:
# Convert to float, set non-convertible to NaN
df_clean['values'] = pd.to_numeric(df_clean['values'], errors='coerce')
# Convert to float, set non-convertible to NaN
df_clean['Raw_values'] = pd.to_numeric(df_clean['Raw_values'], errors='coerce')

Now, apply a proper tranformation:

The `sigmoid_scale()` function normalizes a numeric column by first centering it around its group mean and scaling it by the group standard deviation. It then applies the sigmoid function to transform the normalized values into a **range between 0 and 1**. This is achieved through the expression:

$
\frac{1}{1 + e^{-x}},
$ 

where $ x $ is the normalized value.

In [18]:
def sigmoid_scale(column):
    # Normalize within group first to handle varying scales
    normalized = (column - column.mean()) / column.std()
    return 1 / (1 + np.exp(-normalized))

df_clean['Scaled_Values'] = (
    df_clean
    .groupby('Group_id')['values']  # Group-wise scaling to not distort the analysis
    .transform(sigmoid_scale)
)

The main **advantage of sigmoid scaling** over a more common method like min-max scaling is that it does not map the minimum and maximum values to fixed endpoints (0 and 1), which prevents extreme values from being compressed at the bounds and reduces the disproportionate (non)influence of bottom and top values on downstream metrics, preserving relative differences and producing stable synthesis scores.

- Without going too much into details, robust/standard/logarithm scaling are less convenient: these transforms do not by themselves produce fixed 0–1 bounds across groups, so an extra rescaling step is required to convert outputs to a [0,1] range (sigmoid scaling allows to keep a simple scoring workflow).

In [19]:
df_clean.head(3)

,Group_id,Year,class_name,indicator_summary,class_num,Subregion_code,Subregion_name,State_code,State_name,values,Raw_values,Scaled_Values
0,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,FJ,Fiji,561024940.0000,561024940.0000,0.8507
1,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,PG,Papua New Guinea,572455900.0000,572455900.0000,0.8581
2,1,2022,Political Leadership and Regionalism,Total development aid,1,MEL,Melanesia,SB,Solomon Islands,256630130.0000,256630130.0000,0.5367


## <a id='toc1_6_'></a>[Section 5) Additional features](#toc0_)

### <a id='toc1_6_1_'></a>[Metrics for completeness and contributions/rankings](#toc0_)

Now that the dataset is structured and clean. Let's add some analytical features. This includes metrics related to data completeness, group and class contributions, and ranking information across various domains.

- **Missing values:**

  1) Compute percent-missing values:
    Calculates the percentage of missing `Raw_values` for each combination of `Year`, `class_name`, and `State_name`, storing it in: `pct_missing_values`. This helps identify groups with data gaps.

  2) Flag missing entries:
    A copy of `df_clean` is created as `df_final`, and a new column `Is_Missing` indicates whether each `Raw_values` entry is missing (1 if missing, 0 otherwise).
  
  3) Compute missing value shares:
    The code calculates the count and percentage of missing values per `State` within each `Year` and `class_name`. This information is useful for assessing data quality by merging the missing counts back into `df_final`.

- **Group weights:**

  3) Compute total group values and item shares:
    The code aggregates Scaled_Values to get `Total_Group_Value` for each `Group_id`, `Year`, and `class_name`. It then calculates the share of each item within its group as `Share_Item`, providing insight into the relative contribution of individual items to their respective groups.

  4) Compute class values and related shares:
    Similar aggregation is done at the class level to compute `Total_Class_Value`. The code calculates the share of each group within its class (`Share_Group`) and determines the overall contribution of items to their class (`value_contribution_to_class`) along with the share of each item within the class (`Share_Class`).

- **Ranking:** 
  
  5) Items and states:
    Items are ranked within each `Group` and `Year` based on their `Scaled_Values` using `Item_Rank`. State-level totals are computed and ranked across `Year` and `class_name`, creating `State_rank_per_class`, allowing for comparative assessments of state performance.

  6) Ranking subregions:
    A similar ranking process is applied to subregions based on total scaled values, determining the `Subregion_Class_Rank`.

- **Output:**

  8) Final order of columns:
    `df_final` is restructured to include only relevant columns in a specific order.

In [20]:
# compute percent‐missing per Year, class_name, State_name
df_clean['pct_missing_values'] = (
    df_clean
    .groupby(['Year', 'class_name', 'State_name'])['Raw_values']
    .transform(lambda col: col.isna().mean() * 100)
)
# 1) Copy & flag missing
df_final = df_clean.copy()
df_final['Is_Missing'] = df_final['Raw_values'].isna().astype(int)

# 2) Compute Total_Group_Value and Share_Item
group_totals = (
    df_final
    .groupby(['Group_id', 'Year', 'class_name'])['Scaled_Values']
    .sum()
    .reset_index(name='Total_Group_Value')
)
df_final = df_final.merge(
    group_totals, on=['Group_id', 'Year', 'class_name'], how='left')
df_final['Share_Item'] = df_final['Scaled_Values'] / \
    df_final['Total_Group_Value']

# 3) Compute Total_Class_Value and related shares
class_totals = (
    df_final
    .groupby(['Year', 'class_name'])['Scaled_Values']
    .sum()
    .reset_index(name='Total_Class_Value')
)
df_final = df_final.merge(class_totals, on=['Year', 'class_name'], how='left')
df_final['Share_Group'] = df_final['Total_Group_Value'] / \
    df_final['Total_Class_Value']
df_final['value_contribution_to_class'] = df_final['Share_Item'] * \
    df_final['Share_Group']
df_final['Share_Class'] = df_final['Scaled_Values'] / \
    df_final['Total_Class_Value']

# 4) Rank items within each (Group × Year)
df_final['Item_Rank'] = (
    df_final
    .groupby(['Group_id', 'Year'])['Scaled_Values']
    .rank(ascending=False, method='min')
    .astype("Int64")
)

# 5) Rank each STATE by total Scaled_Values within (Year × class_name)
state_totals = (
    df_final
    .groupby(['Year', 'class_name', 'State_code', 'State_name'])['Scaled_Values']
    .sum()
    .reset_index(name='State_Class_Total')
)
state_totals['State_rank_per_class'] = (
    state_totals
    .groupby(['Year', 'class_name'])['State_Class_Total']
    .rank(ascending=False, method='min')
    .astype("Int64")
)
df_final = df_final.merge(
    state_totals[
        ['Year', 'class_name', 'State_code',
            'State_Class_Total', 'State_rank_per_class']
    ],
    on=['Year', 'class_name', 'State_code'],
    how='left'
)

# 6) Rank each SUBREGION by total Scaled_Values within (Year × class_name)
subregion_totals = (
    df_final
    .groupby(['Year', 'class_name', 'Subregion_code', 'Subregion_name'])['Scaled_Values']
    .sum()
    .reset_index(name='Subregion_Class_Total')
)
subregion_totals['Subregion_Class_Rank'] = (
    subregion_totals
    .groupby(['Year', 'class_name'])['Subregion_Class_Total']
    .rank(ascending=False, method='min')
    .astype("Int64")
)
df_final = df_final.merge(
    subregion_totals[
        ['Year', 'class_name', 'Subregion_code',
            'Subregion_Class_Total', 'Subregion_Class_Rank']
    ],
    on=['Year', 'class_name', 'Subregion_code'],
    how='left'
)

# 7) Compute missing‐value shares per state within each (Year × class_name)
state_missing = (
    df_final
    .groupby(['Year', 'class_name', 'State_code', 'State_name'])['Is_Missing']
    .sum()
    .reset_index(name='State_Missing_Count')
)
class_missing = (
    df_final
    .groupby(['Year', 'class_name'])['Is_Missing']
    .sum()
    .reset_index(name='Class_Missing_Count')
)
state_missing = state_missing.merge(
    class_missing, on=['Year', 'class_name'], how='left'
)
state_missing['State_Missing_Pct'] = (
    100 * state_missing['State_Missing_Count']
    / state_missing['Class_Missing_Count']
).fillna(0)

df_final = df_final.merge(
    state_missing[
        ['Year', 'class_name', 'State_code',
         'State_Missing_Count', 'Class_Missing_Count', 'State_Missing_Pct']
    ],
    on=['Year', 'class_name', 'State_code'],
    how='left'
)

# 8) Final column ordering
final_cols = [
    'Group_id', 'Year', 'class_name', 'class_num', 'indicator_summary',
    'Subregion_code', 'Subregion_name', 'State_code', 'State_name',
    'Raw_values', 'Scaled_Values', 'value_contribution_to_class', 'State_rank_per_class',
    'pct_missing_values'
]
df_final = df_final[final_cols]

df_final.head(3)

,Group_id,Year,class_name,class_num,indicator_summary,Subregion_code,Subregion_name,State_code,State_name,Raw_values,Scaled_Values,value_contribution_to_class,State_rank_per_class,pct_missing_values
0,1,2022,Political Leadership and Regionalism,1,Total development aid,MEL,Melanesia,FJ,Fiji,561024940.0000,0.8507,0.0305,6,0.0000
1,1,2022,Political Leadership and Regionalism,1,Total development aid,MEL,Melanesia,PG,Papua New Guinea,572455900.0000,0.8581,0.0308,12,0.0000
2,1,2022,Political Leadership and Regionalism,1,Total development aid,MEL,Melanesia,SB,Solomon Islands,256630130.0000,0.5367,0.0193,13,0.0000


### <a id='toc1_6_2_'></a>[Interactive report about data quality](#toc0_)

Check missingness and disparity indicators accross the dataset.

Find and store the top N entities (States and Subregions) for:

- Most missing values
- Largest disparities in values
- Most balanced values (smallest disparities)

**Metrics calculation:** the `compute_group_metrics()` function computes metrics for each combination of `class_name`, `Year`, and a specified grouping column. It calculates the number of **missing values**, **maximum** and **minimum** values, and their **disparity** (the difference between maximum and minimum).

**Top N Analysis:** the `top_n_within()` function retrieves the top N entries for each combination of `class_name` and `Year`, based on specified criteria; either the number of missing values or the (large/small) disparity.

In [22]:
VALUE_COL = 'Scaled_Values'
DF = df_final


def compute_group_metrics(df, group_col, value_col):
    grouped = df.groupby(['class_name', 'Year', group_col])[value_col]
    metrics = grouped.agg(
        missing_count=lambda s: int(s.isna().sum()),
        max_val=lambda s: np.nanmax(
            s.values) if s.dropna().size > 0 else np.nan,
        min_val=lambda s: np.nanmin(
            s.values) if s.dropna().size > 0 else np.nan
    ).reset_index()
    metrics['disparity'] = metrics['max_val'] - metrics['min_val']
    return metrics


def top_n_within(groups_df, by, n=5, ascending=False):
    sorted_df = groups_df.sort_values(by=by, ascending=ascending)
    return sorted_df.groupby(['class_name', 'Year']).head(n)


# Pre-compute metrics for both granularities
country_metrics = compute_group_metrics(DF, 'State_name', VALUE_COL)
subregion_metrics = compute_group_metrics(DF, 'Subregion_name', VALUE_COL)

# --- Widget controls ---
granularity_w = widgets.ToggleButtons(
    options=[('State', 'State'), ('Subregion', 'Subregion')],
    value='State'
    
)

# Dynamic options for class_name and Year derived from data
class_options = sorted(DF['class_name'].dropna().unique().tolist())
year_options = sorted(DF['Year'].dropna().unique().tolist())

class_w = widgets.Dropdown(options=class_options, description='class_name:')
year_w = widgets.Dropdown(options=year_options, description='Year:')

top_n_w = widgets.IntSlider(value=5, min=1, max=20,
                            step=1, description='Top N:')

# --- Display / update logic ---
out = widgets.Output(layout={'border': '1px solid gray'})


def render_report(granularity, class_name, year, top_n):
    out.clear_output()
    # Choose metrics df
    if granularity == 'State':
        metrics_df = country_metrics
        group_col = 'State_name'
    else:
        metrics_df = subregion_metrics
        group_col = 'Subregion_name'
    # Filter for selected class_name and year
    filt = (metrics_df['class_name'] == class_name) & (
        metrics_df['Year'] == year)
    df_sel = metrics_df.loc[filt].copy()
    if df_sel.empty:
        with out:
            display(HTML(
                f"<b>No data for class_name={class_name}, Year={year}, granularity={granularity}.</b>"))
        return

    # Compute top lists
    top_missing = df_sel.sort_values(
        by='missing_count', ascending=False).head(top_n)
    top_disparity = df_sel.sort_values(
        by='disparity', ascending=False).head(top_n)
    top_balanced = df_sel.sort_values(
        by='disparity', ascending=True).head(top_n)

    # Add rank columns for clarity
    top_missing = top_missing.assign(rank_missing=range(
        1, len(top_missing)+1)).set_index('rank_missing')
    top_disparity = top_disparity.assign(rank_disparity=range(
        1, len(top_disparity)+1)).set_index('rank_disparity')
    top_balanced = top_balanced.assign(rank_balanced=range(
        1, len(top_balanced)+1)).set_index('rank_balanced')

    with out:
        display(HTML(
            f"<h3>Granularity: {granularity} — class_name: {class_name} — Year: {year}</h3>"))
        display(HTML("<b>Top by missing_count (most missing)</b>"))
        display(top_missing[[group_col, 'missing_count',
                'max_val', 'min_val', 'disparity']])
        display(HTML("<b>Top by disparity (largest disparity)</b>"))
        display(top_disparity[[group_col, 'missing_count',
                'max_val', 'min_val', 'disparity']])
        display(HTML("<b>Top by disparity (smallest = most balanced)</b>"))
        display(top_balanced[[group_col, 'missing_count',
                'max_val', 'min_val', 'disparity']])

# Wire up interactions
def on_change(change):
    render_report(granularity_w.value, class_w.value,
                  year_w.value, top_n_w.value)


granularity_w.observe(on_change, names='value')
class_w.observe(on_change, names='value')
year_w.observe(on_change, names='value')
top_n_w.observe(on_change, names='value')

controls = widgets.HBox([granularity_w, class_w, year_w, top_n_w])
display(controls)
display(out)

# Render
render_report(granularity_w.value, class_w.value, year_w.value, top_n_w.value)

Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

### <a id='toc1_6_3_'></a>[Export data to JSON](#toc0_)

The processed data is now prepared for its integration with the [dataWave app](https://github.com/brooks-code/blue-pacific-dataviz) backend.

![Demo Image](<img/data_pearls.gif> "Screenshot of the dataWave app.")

[Live demo](https://brooks-code.github.io/blue-pacific-dataviz/)

In [23]:
df_final.columns = df_final.columns.str.lower()

# export to JSON file
df_final.to_json('dataWave.json', orient='records', lines=False, force_ascii=False)

Done in the **Summer of 25 [🌠](https://en.wikipedia.org/wiki/Perseids)** - (kudos). 

To the moon 🌑 (and back).

A notebook by [brk](github.com/brooks-code). 